# CREAR UNA BASE DE DATOS EN HIVE METAESTORE USANDO PYSPARK

## 1.	CREAR UNA SPARKSESION

In [ ]:
from pyspark.sql import SparkSession
# Crear SparkSession con soporte para Hive
spark = SparkSession.builder \
    .appName("Ejemplo Hive Metastore") \
    .enableHiveSupport() \
    .getOrCreate()

## 2.	CREAR LA BASE DE DATOS

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS pyspark_hive")
spark.sql("SHOW DATABASES").show()

spark.sql("""
    CREATE TABLE IF NOT EXISTS pyspark_hive.empleado (
        ID INT,
        NOMBRE STRING,
        TELEFONO STRING,
        CORREO STRING,
        FECHA_INGRESO DATE,
        EDAD INT,
        SALARIO DOUBLE,
        ID_EMPRESA INT
    )
    STORED AS PARQUET
""")
